In [1]:
import os
import cv2
import numpy as np

In [2]:
dirs = ['ants', 'bees']
pixels = []
labels = []

for i, d in enumerate(dirs):
    files = os.listdir('../datasets/ric/' + d)
    
    for f in files:
        img = cv2.imread('../datasets/ric/' + d + '/' + f, 0)
        img = cv2.resize(img, (128, 128))
        img = np.array(img).flatten().tolist()
        pixels.append(img)
        
        labels.append(i)

In [3]:
import pandas as pd

pixels_df = pd.DataFrame(pixels)
pixels_df = pixels_df/255

labels_df = pd.DataFrame(labels)
labels_df = labels_df.rename(columns={0: 'label'})

img_set = pd.concat([pixels_df, labels_df], axis=1)
img_set.head()

,0,1,2,3,4,5,6,7,8,9,...,16375,16376,16377,16378,16379,16380,16381,16382,16383,label
0,0.549020,0.552941,0.545098,0.552941,0.545098,0.556863,0.556863,0.556863,0.552941,0.556863,...,0.294118,0.282353,0.286275,0.274510,0.294118,0.286275,0.286275,0.278431,0.360784,0
1,0.756863,0.831373,0.894118,0.815686,0.862745,0.866667,0.737255,0.772549,0.768627,0.219608,...,0.866667,0.890196,0.933333,0.905882,0.882353,0.921569,0.945098,0.858824,0.823529,0
2,0.941176,0.917647,0.945098,0.933333,0.952941,0.886275,0.941176,0.788235,0.882353,0.933333,...,0.717647,0.552941,0.729412,0.686275,0.239216,0.352941,0.443137,0.462745,0.596078,0
3,0.537255,0.533333,0.521569,0.545098,0.552941,0.564706,0.580392,0.592157,0.600000,0.623529,...,0.462745,0.454902,0.470588,0.474510,0.466667,0.466667,0.470588,0.458824,0.470588,0
4,0.521569,0.545098,0.549020,0.564706,0.572549,0.592157,0.588235,0.584314,0.580392,0.568627,...,0.537255,0.564706,0.674510,0.541176,0.482353,0.800000,0.894118,0.611765,0.654902,0


## モルフォロジー変換

In [4]:
import matplotlib.pyplot as plt
%matplotlib

img = cv2.imread('../datasets./ric/ants/Swiss-army-ant.jpg', 0)
ret, bin_img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)

kernel = np.ones((3,3), np.uint8)
img_el = cv2.erode(bin_img, kernel, iterations=1)


Using matplotlib backend: Qt5Agg


## 収縮 erosion

In [5]:
plt.imshow(img_el, cmap='gray')
plt.show()

## 膨張 Dilation

In [6]:
img_dl = cv2.dilate(bin_img, kernel, iterations=1)
plt.imshow(img_dl, cmap='gray')
plt.show()

## オープニング opening

In [7]:
img_op = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, kernel)
plt.imshow(img_op, cmap='gray')

## クロージング Closing

In [8]:
img_cl = cv2.morphologyEx(bin_img, cv2.MORPH_CLOSE, kernel)
plt.imshow(img_cl, cmap='gray')

In [9]:
hist_gr, bins = np.histogram(img.ravel(), 256, [0,256])

plt.xlim(0, 255)
plt.plot(hist_gr, '-r')
plt.xlabel('pixel value')
plt.ylabel('number of pixels')
plt.show()

## t-SNEによる次元圧縮

In [10]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
pixels_tsne = tsne.fit_transform(pixels_df)

print(pixels_df.shape)
print(pixels_tsne.shape)

img_set_tsne = pd.concat([pd.DataFrame(pixels_tsne), labels_df], axis=1)
img_set_tsne.head()

(397, 16384)
(397, 2)


,0,1,label
0,2.033107,2.450779,0
1,4.387461,7.933300,0
2,6.118368,8.837999,0
3,-2.117239,-2.292316,0
4,1.787750,4.625388,0


In [12]:
img_set_tsne_0 = img_set_tsne[img_set_tsne['label'] == 0]
img_set_tsne_0 = img_set_tsne_0.drop('label', axis=1)
plt.scatter(img_set_tsne_0[0], img_set_tsne_0[1], c='red', label=0)

img_set_tsne_1 = img_set_tsne[img_set_tsne['label'] == 1]
img_set_tsne_1 = img_set_tsne_1.drop('label', axis=1)
plt.scatter(img_set_tsne_1[0], img_set_tsne_1[1], c='blue', label=1)

plt.xlabel('1st-comp')
plt.ylabel('2nd-comp')
plt.legend()
plt.grid()
plt.show()